
# Pig Detection with YOLOv8 (Training from Scratch)

This notebook prepares the TAICa pig detection dataset for YOLOv8, trains a detector **from random initialization (no pretrained weights)**, evaluates it, and exports predictions that follow the submission format described in the homework slides.



**Setup Notes**
- Keep the original dataset folder `taica-cvpdl-2025-hw-1` in the homework root (three levels up from this notebook).
- Review `sample_submission.csv` to verify the expected `PredictionString` layout.


In [8]:

# If running in a clean environment, uncomment the line below to install dependencies.
# %pip install ultralytics==8.1.0


In [9]:

import csv
import random
import shutil
from collections import defaultdict
from datetime import datetime
from pathlib import Path
from typing import Dict, List, Tuple

import numpy as np
from PIL import Image
import torch
from ultralytics import YOLO

torch.set_float32_matmul_precision('high')
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {DEVICE}')


Using device: cpu


In [10]:

PROJECT_DIR = Path.cwd()
DATA_ROOT = (PROJECT_DIR / '../../../taica-cvpdl-2025-hw-1').resolve()
TRAIN_IMG_DIR = DATA_ROOT / 'train' / 'img'
TEST_IMG_DIR = DATA_ROOT / 'test' / 'img'
GT_PATH = DATA_ROOT / 'train' / 'gt.txt'

if not DATA_ROOT.exists():
    raise FileNotFoundError(f'Dataset folder not found at {DATA_ROOT}. Update DATA_ROOT if your layout differs.')

EXPERIMENT_NAME = 'pig_detection_yolo'
RUN_ID = datetime.now().strftime('%Y%m%d-%H%M%S')
EXPERIMENT_DIR = (PROJECT_DIR / 'artifacts' / EXPERIMENT_NAME / RUN_ID).resolve()
YOLO_DATA_DIR = EXPERIMENT_DIR / 'dataset'
RUNS_DIR = EXPERIMENT_DIR / 'runs'

YOLO_DATA_DIR.mkdir(parents=True, exist_ok=True)
RUNS_DIR.mkdir(parents=True, exist_ok=True)
print(f'Experiment directory: {EXPERIMENT_DIR}')


Experiment directory: /Users/daniel/Desktop/Study/Computer Vision/HW1/hw1_314706007/code_314706007/src/artifacts/pig_detection_yolo/20251010-211459


In [11]:

def load_annotations(gt_path: Path) -> Dict[int, List[Tuple[float, float, float, float]]]:
    annotations: Dict[int, List[Tuple[float, float, float, float]]] = defaultdict(list)
    with gt_path.open('r') as fh:
        for line in fh:
            parts = line.strip().split(',')
            if len(parts) < 5:
                continue
            image_id = int(parts[0])
            x, y, w, h = map(float, parts[1:5])
            annotations[image_id].append((x, y, w, h))
    if not annotations:
        raise RuntimeError(f'No annotations parsed from {gt_path}.')
    return annotations

def convert_bbox_to_yolo(x: float, y: float, w: float, h: float, img_w: int, img_h: int):
    xc = (x + w / 2.0) / img_w
    yc = (y + h / 2.0) / img_h
    ww = w / img_w
    hh = h / img_h
    xc = min(max(xc, 0.0), 1.0)
    yc = min(max(yc, 0.0), 1.0)
    ww = min(max(ww, 0.0), 1.0)
    hh = min(max(hh, 0.0), 1.0)
    return xc, yc, ww, hh

def prepare_dataset(images_dir: Path, annotations: Dict[int, List[Tuple[float, float, float, float]]], output_dir: Path, val_ratio: float = 0.1, seed: int = 42):
    if output_dir.exists():
        shutil.rmtree(output_dir)
    for split in ['train', 'val']:
        (output_dir / 'images' / split).mkdir(parents=True, exist_ok=True)
        (output_dir / 'labels' / split).mkdir(parents=True, exist_ok=True)

    all_ids = sorted(annotations.keys())
    available_ids = []
    missing_ids = []
    for image_id in all_ids:
        img_name = f'{image_id:08d}.jpg'
        if (images_dir / img_name).exists():
            available_ids.append(image_id)
        else:
            missing_ids.append(image_id)

    if missing_ids:
        sample = ', '.join(f'{mid:08d}' for mid in missing_ids[:5])
        print(f'Skipping {len(missing_ids)} annotation entries without images. Examples: {sample}')

    rng = random.Random(seed)
    rng.shuffle(available_ids)
    val_count = max(1, int(len(available_ids) * val_ratio)) if available_ids else 0
    val_ids = set(available_ids[:val_count])

    stats = {'train': 0, 'val': 0, 'boxes': 0, 'skipped': len(missing_ids)}

    for image_id in available_ids:
        split = 'val' if image_id in val_ids else 'train'
        img_name = f'{image_id:08d}.jpg'
        src_path = images_dir / img_name
        with Image.open(src_path) as img:
            img_w, img_h = img.size

        dst_img = output_dir / 'images' / split / img_name
        shutil.copy2(src_path, dst_img)

        label_lines = []
        for bbox in annotations[image_id]:
            xc, yc, ww, hh = convert_bbox_to_yolo(*bbox, img_w=img_w, img_h=img_h)
            label_lines.append(f'0 {xc:.6f} {yc:.6f} {ww:.6f} {hh:.6f}')
        label_path = output_dir / 'labels' / split / f'{image_id:08d}.txt'
        label_path.write_text('\n'.join(label_lines))
        stats[split] += 1
        stats['boxes'] += len(label_lines)

    return stats


In [12]:

annotations = load_annotations(GT_PATH)
stats = prepare_dataset(TRAIN_IMG_DIR, annotations, YOLO_DATA_DIR, val_ratio=0.1, seed=SEED)
print(f'Dataset prepared at {YOLO_DATA_DIR}')
print(f"train images: {stats['train']} | val images: {stats['val']} | total boxes: {stats['boxes']}")


Skipping 4 annotation entries without images. Examples: 00000604, 00000605, 00000606, 00000607
Dataset prepared at /Users/daniel/Desktop/Study/Computer Vision/HW1/hw1_314706007/code_314706007/src/artifacts/pig_detection_yolo/20251010-211459/dataset
train images: 1140 | val images: 126 | total boxes: 38619


In [13]:

DATA_CONFIG_PATH = EXPERIMENT_DIR / 'pig_dataset.yaml'
config_lines = [
    f'path: {YOLO_DATA_DIR.as_posix()}',
    'train: images/train',
    'val: images/val',
    'nc: 1',
    'names:',
    '  0: pig',
]
DATA_CONFIG_PATH.write_text('\n'.join(config_lines) + '\n')
print(f'YOLO data config saved to {DATA_CONFIG_PATH}')


YOLO data config saved to /Users/daniel/Desktop/Study/Computer Vision/HW1/hw1_314706007/code_314706007/src/artifacts/pig_detection_yolo/20251010-211459/pig_dataset.yaml


In [ ]:

MODEL_CONFIG = 'yolov8n.yaml'
EPOCHS = 100
BATCH_SIZE = 16
IMAGE_SIZE = 640
RUN_NAME = 'yolov8n_scratch'

model = YOLO(MODEL_CONFIG)
train_results = model.train(
    data=str(DATA_CONFIG_PATH),
    epochs=EPOCHS,
    imgsz=IMAGE_SIZE,
    batch=BATCH_SIZE,
    device=0 if torch.cuda.is_available() else 'cpu',
    project=str(RUNS_DIR),
    name=RUN_NAME,
    pretrained=False,
    seed=SEED,
    verbose=True,
)
train_results


Ultralytics 8.3.209 🚀 Python-3.11.11 torch-2.8.0 CPU (Apple M2)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/Users/daniel/Desktop/Study/Computer Vision/HW1/hw1_314706007/code_314706007/src/artifacts/pig_detection_yolo/20251010-211459/pig_dataset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8n_scratch, nbs=64, nms=False, opset

In [ ]:

val_metrics = model.val(
    data=str(DATA_CONFIG_PATH),
    imgsz=IMAGE_SIZE,
    conf=0.001,
    iou=0.6,
    project=str(RUNS_DIR),
    name=f'{RUN_NAME}_val',
)
val_metrics.results_dict


In [ ]:

best_weights = RUNS_DIR / RUN_NAME / 'weights' / 'best.pt'
if not best_weights.exists():
    raise FileNotFoundError(f'Did not find trained weights at {best_weights}. Check the training output above.')

predictor = YOLO(best_weights)
predictions = predictor.predict(
    source=str(TEST_IMG_DIR),
    imgsz=IMAGE_SIZE,
    conf=0.001,
    iou=0.6,
    device=0 if torch.cuda.is_available() else 'cpu',
    verbose=True,
)

submission_rows = []
for result in predictions:
    image_id = Path(result.path).stem.lstrip('0') or '0'
    boxes = result.boxes
    if boxes is None or boxes.xyxy is None or boxes.xyxy.shape[0] == 0:
        submission_rows.append((image_id, ''))
        continue
    xyxy = boxes.xyxy.cpu().numpy()
    scores = boxes.conf.cpu().numpy()
    classes = boxes.cls.cpu().numpy().astype(int)
    entries = []
    for (x1, y1, x2, y2), score, cls_id in zip(xyxy, scores, classes):
        width = x2 - x1
        height = y2 - y1
        entries.append(f'{score:.6f} {x1:.2f} {y1:.2f} {width:.2f} {height:.2f} {cls_id}')
    submission_rows.append((image_id, ' '.join(entries)))

submission_path = EXPERIMENT_DIR / 'submission.csv'
with submission_path.open('w', newline='') as fh:
    writer = csv.writer(fh)
    writer.writerow(['Image_ID', 'PredictionString'])
    writer.writerows(submission_rows)
print(f'Saved submission to {submission_path}')
submission_rows[:5]
